In [3]:
from PIL import Image
import numpy as np
c0 = Image.open('data/test/0/1.png')
display(c0)
c0_numpy = np.array(c0)
c0_numpy.shape

(28, 28, 4)

In [10]:
c0_numpy[:, :, :-1].shape # 투명도 제거

(28, 28, 3)

# 1. CNN

In [15]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from PIL import Image
np.random.seed(3)

In [17]:
circle = Image.open('data/handwriting/handwriting_shape/train/circle/circle001.png')
np.array(circle).shape
# 모든 이미지의 shape이 같은 필요는 없음

(24, 24, 4)

In [22]:
# 1. Dataset
# Train: data/handwriting/handwriting_shape/train/*
# Test: data/handwriting/handwriting_shape/test/*

# 이미지 증강 변수 생성
train_data_gen = ImageDataGenerator(rescale = 1./255) # 각 픽셀 값이 0~1의 값으로 정규화

# flow_from_directory
# directory의 바로 하위 디렉토리 이름을 레이블이라고 간주 (circle, rectangle, triangle)
# 하위 디렉토리 내 파일들을 해당 레이블의 이미지들이라고 알아서 추측 
train_data = train_data_gen.flow_from_directory(
    'data/handwriting/handwriting_shape/train', # 디렉토리 경로
    target_size = (24, 24),     # Numpy Array의 이미지 크기 조정
    class_mode = 'categorical', # One-Hot Encoding 형식으로 반환(1 0 0 / 0 1 0 / 0 0 1)
    batch_size = 3              # 한 번에 반환할 이미지와 label 수
)
test_data_gen = ImageDataGenerator(rescale = 1./255)
test_data = train_data_gen.flow_from_directory(
    'data/handwriting/handwriting_shape/test',
    target_size = (24, 24),
    class_mode = 'categorical',
    batch_size = 3,
    shuffle = False, # True(기본값), 이미지와 label 순서가 고정
)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [23]:
test_data

In [26]:
X_images, Y_labels = next(test_data) # 한 batch size만큼만 데이터 가져오기
X_images, Y_labels

(array([[[[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         [[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [0.82745105, 0.82745105, 0.82745105],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         [[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [0.5882353 , 0.5882353 , 0.5882353 ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         ...,
 
         [[1.        , 1.        , 1.        ],
          [1.     

In [27]:
print('데이터 전체 갯수:', test_data.n)
print('batch size:', test_data.batch_size)

데이터 전체 갯수: 15
batch size: 3


In [30]:
# 전체 가져오기
all_images = [] # 독립변수
all_labels = [] # 종속변수

for images, labels in test_data:
    all_images.append(images)
    all_labels.append(labels)
    
    if len(all_images)*test_data.batch_size >= test_data.n:
        print('끝')
        break
X_test_images = np.concatenate(all_images)
Y_test_labels = np.concatenate(all_labels)

X_test_images.shape, Y_test_labels.shape

끝


((15, 24, 24, 3), (15, 3))

In [31]:
Y_test_labels

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [40]:
# 2. Model 구성
model = Sequential()
model.add(Conv2D(16, kernel_size = (3, 3), activation = 'relu', input_shape = (24, 24, 3)))
model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten()) # 3차원 → 1차원
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 3, activation = 'softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 22, 22, 16)        448       
                                                                 
 conv2d_6 (Conv2D)           (None, 20, 20, 32)        4640      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 3200)              0         
                                                                 
 dense_4 (Dense)             (None, 64)                204864    
                                                                 
 dense_5 (Dense)             (None, 3)                 195       
                                                      

In [41]:
# 3. Training Process Setting
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
# 4. Training
model.fit(train_data, 
          epochs = 50, 
          validation_data = test_data)

Epoch 1/50
15/15 [==============================] - 1s 24ms/step - loss: 1.1576 - accuracy: 0.3556 - val_loss: 0.9563 - val_accuracy: 0.3333
Epoch 2/50
15/15 [==============================] - 0s 10ms/step - loss: 0.7260 - accuracy: 0.8222 - val_loss: 0.5389 - val_accuracy: 1.0000
Epoch 3/50
15/15 [==============================] - 0s 9ms/step - loss: 0.3556 - accuracy: 0.9111 - val_loss: 0.4025 - val_accuracy: 0.9333
Epoch 4/50
15/15 [==============================] - 0s 10ms/step - loss: 0.1267 - accuracy: 0.9778 - val_loss: 0.1224 - val_accuracy: 1.0000
Epoch 5/50
15/15 [==============================] - 0s 10ms/step - loss: 0.0546 - accuracy: 0.9778 - val_loss: 0.0527 - val_accuracy: 1.0000
Epoch 6/50
15/15 [==============================] - 0s 9ms/step - loss: 0.0127 - accuracy: 1.0000 - val_loss: 0.0367 - val_accuracy: 1.0000
Epoch 7/50
15/15 [==============================] - 0s 9ms/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 0.0291 - val_accuracy: 1.0000
Epoch 8/50
15/15

In [42]:
# Model 평가
loss, accuracy = model.evaluate(test_data)
print('loss:', loss)
print('accuracy:', accuracy)

5/5 [==============================] - 0s 6ms/step - loss: 0.0048 - accuracy: 1.0000
loss: 0.004803040064871311
accuracy: 1.0


In [43]:
# prediction
yhat = model.predict(test_data)
yhat.argmax(axis = 1)

5/5 [==============================] - 0s 5ms/step


array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2], dtype=int64)

In [44]:
# 실제 데이터
test_data.labels

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])

In [46]:
# format 변경
np.set_printoptions(formatter = {'float':lambda x: "{:02f}".format(x)})
yhat

array([[0.999116, 0.000874, 0.000011],
       [0.999995, 0.000005, 0.000000],
       [0.999681, 0.000172, 0.000147],
       [0.999980, 0.000020, 0.000000],
       [0.999952, 0.000047, 0.000000],
       [0.000005, 0.999985, 0.000009],
       [0.000117, 0.999775, 0.000108],
       [0.000031, 0.999925, 0.000044],
       [0.000020, 0.999972, 0.000007],
       [0.013674, 0.974722, 0.011604],
       [0.000002, 0.000139, 0.999858],
       [0.000000, 0.000273, 0.999727],
       [0.000002, 0.035076, 0.964921],
       [0.000000, 0.000019, 0.999980],
       [0.000009, 0.008634, 0.991358]], dtype=float32)

# 2. 데이터 증강을 통한 CNN accuracy 올리기
- 데이터 추가 확보
- 이미지 데이터를 증강(데이터 부풀리기): Only Training Dataset
- Layer 증가(Conv2D, MaxPooling, Batch Normalization, Dense)
- Activation Function, Optimizer, Dropout(과적합 다소 해결) 

In [50]:
# 1. Dataset
train_data_gen = ImageDataGenerator(rescale = 1.0/255,        # scale 조정 
                                    rotation_range = 10,     # 이미지를 최대 +10~-10도 회전
                                    width_shift_range = .2,  # 가로 방향으로 최대 가로 크기의 20% 이동
                                    height_shift_range = .2, # 세로 방향으로 최대 세로 크기의 20% 이동
                                    shear_range = .7,        # 최대 0.7radian(40도)으로 이미지 밀림
                                    zoom_range = [.9, 1.5],  # 90~150% 크기로 확대/축소
                                    fill_mode = 'nearest',   # 이미지 확대 시 채우는 방식
                                    horizontal_flip = True,  # 수평 방향 뒤집기
                                    vertical_flip  = True, # 수직 방향 뒤집기
                                   )
train_data = train_data_gen.flow_from_directory(
    'data/handwriting/handwriting_shape/train',
    target_size = (24, 24),
    class_mode = 'categorical',
    batch_size = 3
)
test_data_gen = ImageDataGenerator(rescale = 1./255)
test_data = test_data_gen.flow_from_directory(
    'data/handwriting/handwriting_shape/test',
    target_size = (24, 24),
    class_mode = 'categorical',
    batch_size = 3,
    # shuffle = True # 기본값
)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [69]:
# 2. Model 구성
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu', input_shape = (24, 24, 3)))
model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(.2))

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu', input_shape = (24, 24, 3)))
model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(.2))

model.add(Flatten()) # 3차원 → 1차원
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 3, activation = 'softmax'))
# model.summary()
# 3. Training Process Setting
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
# 4. Training
model.fit(train_data, 
          epochs = 50, 
          validation_data = test_data)

Epoch 1/50
15/15 [==============================] - 1s 28ms/step - loss: 1.1496 - accuracy: 0.2667 - val_loss: 1.0961 - val_accuracy: 0.3333
Epoch 2/50
15/15 [==============================] - 0s 14ms/step - loss: 1.1095 - accuracy: 0.2889 - val_loss: 1.0909 - val_accuracy: 0.6667
Epoch 3/50
15/15 [==============================] - 0s 14ms/step - loss: 1.1031 - accuracy: 0.3111 - val_loss: 1.0919 - val_accuracy: 0.3333
Epoch 4/50
15/15 [==============================] - 0s 14ms/step - loss: 1.0985 - accuracy: 0.3556 - val_loss: 1.0858 - val_accuracy: 0.3333
Epoch 5/50
15/15 [==============================] - 0s 13ms/step - loss: 1.0920 - accuracy: 0.3333 - val_loss: 1.0774 - val_accuracy: 0.4667
Epoch 6/50
15/15 [==============================] - 0s 15ms/step - loss: 1.0819 - accuracy: 0.4667 - val_loss: 1.0516 - val_accuracy: 0.4000
Epoch 7/50
15/15 [==============================] - 0s 11ms/step - loss: 1.1338 - accuracy: 0.4000 - val_loss: 1.0639 - val_accuracy: 0.5333
Epoch 8/50
15

In [70]:
# Model 평가
score = model.evaluate(test_data)
print("accuracy:", score[1])

5/5 [==============================] - 0s 6ms/step - loss: 0.0111 - accuracy: 1.0000
accuracy: 1.0


In [71]:
pred = model.predict(test_data)
print(pred)

5/5 [==============================] - 0s 4ms/step
[[0.000000 1.000000 0.000000]
 [0.013600 0.000000 0.986400]
 [0.999994 0.000006 0.000000]
 [0.000039 0.000000 0.999961]
 [0.106771 0.000006 0.893223]
 [1.000000 0.000000 0.000000]
 [1.000000 0.000000 0.000000]
 [0.000000 1.000000 0.000000]
 [0.000000 1.000000 0.000000]
 [0.037852 0.000000 0.962147]
 [0.001445 0.000000 0.998555]
 [1.000000 0.000000 0.000000]
 [0.000000 1.000000 0.000000]
 [1.000000 0.000000 0.000000]
 [0.000000 1.000000 0.000000]]


In [72]:
pred.argmax(axis = 1)

array([1, 2, 0, 2, 2, 0, 0, 1, 1, 2, 2, 0, 1, 0, 1], dtype=int64)